# Intrinsic Valuation Functions

In [13]:
# https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html
def country_def_spread(country):
    # AKA bond default spread
    #
    # Three ways
    # 1) government bonds 
    # def_spread = sov bond rate ('currency') - rfr('currency')
    # USD bond rate - USD riskfree rate = def spread
    # poland: .0468 - .0388 = .008
    # 2) find CDS for sovereign emerging government
    # US CDS = .0032
    # poland CDS = .0146
    # poland CDS net of US = .0113
    # 3) Lookup def spread based on rating
    if country=='USA':
        return 0.0
    if country=='Brazil':
        return 0.0368
    if country=='Germany':
        return 0.0
    if country=='Japan':
        return 0.0086
    if country=='Brazil Test':
        return .0368
    
# http://www.worldgovernmentbonds.com/country/
def govt_bond_rate(country):
    if country=='USA':
        return 0.03568
    if country=='Brazil':
        return 0.12604
    if country=='Germany':
        return 0.02481
    if country=='Japan':
        return 0.00461
    if country=='Brazil Test':
        return 0.1276
    
def risk_free_rate(country):
    return govt_bond_rate(country) - country_def_spread(country)

def relative_eq_mkt_vol(country):
    if country=='Brazil Test':
        stdev_index = .3
        stdev_bonds = .2
    
    return stdev_index/stdev_bonds

# https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html
def country_risk_premium(country):
    
    if country=='Mature Embraer Test':
        return .0
    if country=='Brazil Embraer Test':
        return .08
    if country=='Argentina Ambev Test':
        return .09
    if country=='Bolivia Ambev Test':
        return .0488
    if country=='Brazil Ambev Test':
        return .0263
    if country=='Canada Ambev Test':
        return .0
    if country=='Chile Ambev Test':
        return .0105
    if country=='Ecuador Ambev Test':
        return .0675
    if country=='Paraguay Ambev Test':
        return .06
    if country=='Peru Ambev Test':
        return .03
    if country=='Western Europe Coca Cola Test':
        return .0067
    if country=='Eastern Europe Russia Coca Cola Test':
        return .0260
    if country=='Asia Coca Cola Test':
        return .0163
    if country=='Latin America Coca Cola Test':
        return .0342
    if country=='Australia Coca Cola Test':
        return .0
    if country=='Africa Coca Cola Test':
        return .0382
    if country=='North America Coca Cola Test':
        return .0

    return country_def_spread(country)*relative_eq_mkt_vol(country)

def weighted_country_risk_premium(countries, weights):
    if len(countries)!=len(weights):
        raise Exception("Countries and weights must be same length")
    
    # Initialize CRP and add risk components by country
    wcrp=0
    for i,cf in enumerate(countries):
        wcrp += weights[i]*country_risk_premium(countries[i])
    return wcrp

def mature_market_premium():
    # TODO compute/get IERP
    # TODO make this a more generic market_premium()
    return .0594
    
# https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html
def country_equity_risk_premium(countries,weights):
    return mature_market_premium() + weighted_country_risk_premium(countries,weights)

def company_erp(crp_type,country,beta,lambda_risk):

    # country = either country of incorporation/trading OR 
    # list of 
    
    # Method 1: Assume every company in country is equally exposed to country risk
    if crp_type=="equal_country_risk":
        return risk_free_rate(country) + country_risk_premium(country) + beta*mature_market_premium()

    # Method 2: Assume company risk in country is similar to mature market risk
    if crp_type=="constant_beta":
        return risk_free_rate(country) + beta*(mature_market_premium() + country_risk_premium(country))

    # Method 3: Country risk is separate risk factor
    if crp_type=="lambda_factors":
        return risk_free_rate(country) + beta*mature_market_premium() + lambda_risk*country_risk_premium(country)




# TODO - return to slide 54

# def beta(company)

# def erp('market')

# Slide 24
# def coe(rfr,beta,erp):
#     return rfr + beta*erp


# Long term cost to borrow?
def pre_tax_rate(country):
    if country=='test':
        return .1
        
def tax_rate(country):
    if country=='test':
        return .5
    
def cod(country):
    return pre_tax_rate(country)*(1 - tax_rate(country))
    
def wacc(mkt_equity, coe, mkt_debt, codebt):
    return coe*mkt_equity/(mkt_equity + mkt_debt) + codebt*mkt_debt/(mkt_equity + mkt_debt)
    
    
def dcf(rate,flow,term_val):
    if len(rate)!=len(flow):
        raise Exception("Cash flows and rates must be same length")
    
    # Initialize value, add cash flows, add terminal value
    value=0
    for i,cf in enumerate(flow):
        value += flow[i]/(1 + rate[i])**(i+1)
    value += term_val/(1 + rate[len(flow)-1])**(len(flow))
    
    return value

# Country and Currency Parameters

In [31]:
country = 'USA'
currency = 'USD'

# Company Data

In [6]:
# https://pages.stern.nyu.edu/~adamodar/pdfiles/eqnotes/valpacket1spr23.pdf
# Slides 8-10

# Equity test case - cost of equity, CF to equity, equity terminal value
test_coe = .13625
test_coeq = [test_coe, test_coe, test_coe, test_coe, test_coe]
test_eq_flow = [50,60,68,76.2,83.49]
test_eq_tv = 1603.0
test_eq_val = dcf(test_coeq,test_eq_flow,test_eq_tv)

# Firm case - cost of capital, CF to firm, firm terminal value
test_mkt_debt = 800
test_cod = cod('test')
test_coc = wacc(test_eq_val, test_coe, test_mkt_debt, test_cod)
test_cocap = [test_coc, test_coc, test_coc, test_coc, test_coc]
test_firm_flow = [90,100,108,116.2,123.49]
test_firm_tv = 2363.008
test_firm_val = dcf(test_cocap,test_firm_flow,test_firm_tv)

# Corporate risk premium calculations: slides 56+
embraer_countries = ['Mature Embraer Test',
                     'Brazil Embraer Test']
embraer_weights = [.97, .03]

ambev_countries = ['Argentina Ambev Test',
                   'Bolivia Ambev Test',
                   'Brazil Ambev Test',
                   'Canada Ambev Test',
                   'Chile Ambev Test',
                   'Ecuador Ambev Test',
                   'Paraguay Ambev Test',
                   'Peru Ambev Test']
ambev_weights = [.0931, .0196, .6373, .1127, .0343, .0294, .0147, .0588]

cocacola_countries = ['Western Europe Coca Cola Test',
                      'Eastern Europe Russia Coca Cola Test',
                      'Asia Coca Cola Test',
                      'Latin America Coca Cola Test',
                      'Australia Coca Cola Test',
                      'Africa Coca Cola Test',
                      'North America Coca Cola Test']
cocacola_weights = [.19, .05, .15, .15, .04, .04, .4]

# Value Calculations

In [18]:
print(test_eq_val)
print(test_firm_val)
print(risk_free_rate('Brazil Test')) # slide 38, should be .0908
print(country_equity_risk_premium(['Brazil Test'],[1.0])) # slide 51, should be .1146

print(weighted_country_risk_premium(embraer_countries,embraer_weights)) # slide 56, should be .0024
print(weighted_country_risk_premium(ambev_countries,ambev_weights)) # slide 56, should be .0311
print(weighted_country_risk_premium(cocacola_countries,cocacola_weights)) # slide 57, should be .0114

1073.0065063585455
1873.4651511854781
0.09079999999999999
0.1146
0.0024
0.031087120000000003
0.011676
